In [1]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack

/Applications/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
file_path = "data/train.jsonl"

In [3]:
data = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))
df = pd.DataFrame(data)

In [4]:
df_filtered = df[(df["verifiable"] == "VERIFIABLE") & (df["label"].isin(["SUPPORTS", "REFUTES"]))].copy()
df_filtered = df_filtered[["claim", "label"]]

In [6]:
excel_filename = "data/filtered_fever_data.xlsx"
df_filtered.to_excel(excel_filename, index=False)

In [22]:
# Train-test split (70-30 ratio)
X_train, X_test, y_train, y_test = train_test_split(df_filtered["claim"], df_filtered["label"], test_size=0.3, random_state=42)

### 1. TF-IDF Vectorization ###

In [23]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [24]:
# Train and evaluate Naive Bayes model
model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("\n=== TF-IDF Model Performance ===")
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))


=== TF-IDF Model Performance ===
Accuracy: 0.7740946483319673
              precision    recall  f1-score   support

     REFUTES       0.86      0.21      0.33      8987
    SUPPORTS       0.77      0.99      0.86     23956

    accuracy                           0.77     32943
   macro avg       0.81      0.60      0.60     32943
weighted avg       0.79      0.77      0.72     32943



### 2. N-Gram Processing (Unigram + Bigram + Trigram) ###

In [25]:
ngram_vectorizer = TfidfVectorizer(ngram_range=(1,3))  # Includes unigrams, bigrams, and trigrams
X_train_ngram = ngram_vectorizer.fit_transform(X_train)
X_test_ngram = ngram_vectorizer.transform(X_test)

In [26]:
model_ngram = MultinomialNB()
model_ngram.fit(X_train_ngram, y_train)
y_pred_ngram = model_ngram.predict(X_test_ngram)

print("\n=== N-Gram Model Performance ===")
print("Accuracy:", accuracy_score(y_test, y_pred_ngram))
print(classification_report(y_test, y_pred_ngram))


=== N-Gram Model Performance ===
Accuracy: 0.7779194366026166
              precision    recall  f1-score   support

     REFUTES       0.93      0.20      0.33      8987
    SUPPORTS       0.77      0.99      0.87     23956

    accuracy                           0.78     32943
   macro avg       0.85      0.60      0.60     32943
weighted avg       0.81      0.78      0.72     32943



# Merge TF-IDF & N-Gram features

In [29]:
X_train_combined = hstack([X_train_tfidf, X_train_ngram])
X_test_combined = hstack([X_test_tfidf, X_test_ngram])

# Train & Evaluate Naive Bayes Model
model_combined = MultinomialNB()
model_combined.fit(X_train_combined, y_train)
y_pred_combined = model_combined.predict(X_test_combined)

In [30]:
print("\n=== Combined TF-IDF + N-Gram Model Performance ===")
print("Accuracy:", accuracy_score(y_test, y_pred_combined))
print(classification_report(y_test, y_pred_combined))


=== Combined TF-IDF + N-Gram Model Performance ===
Accuracy: 0.7830495097592812
              precision    recall  f1-score   support

     REFUTES       0.90      0.23      0.37      8987
    SUPPORTS       0.77      0.99      0.87     23956

    accuracy                           0.78     32943
   macro avg       0.84      0.61      0.62     32943
weighted avg       0.81      0.78      0.73     32943

